In [1]:
!pip install qwen_vl_utils
!pip install transformers
!pip install datasets
import torch
from transformers import (
    Qwen2VLForConditionalGeneration, AutoProcessor,
    Trainer, TrainingArguments
)
from qwen_vl_utils import process_vision_info
from datasets import load_dataset, Dataset
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
from PIL import Image
import torch
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
compute = True

# Check device compatibility
device = "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"

print('device = ', device)
model_name = "Qwen/Qwen2-VL-2B-Instruct"

img_folder_path = "/content/drive/MyDrive/MATH-V-main"

dataset_name = "MathLLMs/MathVision"

file_name = f"data/output/{dataset_name.split('/')[-1]}_{model_name.split('/')[-1]}.csv"



device =  cpu


In [19]:
# Load the dataset
data_list = load_dataset(dataset_name, split='test')


# Split data into training and validation sets
train_size = int(0.7 * len(data_list))
val_size = int(0.1 * len(data_list))
test_size = len(data_list) - train_size - val_size
data_list = data_list.shuffle(seed=42)  # Shuffle the dataset for randomness

train_data = data_list.select(range(train_size))
val_data = data_list.select(range(train_size, train_size + val_size))
test_data = data_list.select(range(train_size+val_size, len(data_list)))

# test_size = 100
# # Split into train and test
# test_dataset = Dataset.from_dict(
#     train_dataset[-test_size:]
# )

# train_dataset = Dataset.from_dict(
#     train_dataset[:-test_size]
# )

In [8]:
test_data[0]

{'id': '2449',
 'question': 'Triangle $ ABC$ is a right triangle with $ \\angle ACB$ as its right angle, $ m\\angle ABC = 60^\\circ$, and $ AB = 10$. Let $ P$ be randomly chosen inside $ \\triangle ABC$, and extend $ \\overline{BP}$ to meet $ \\overline{AC}$ at $ D$. What is the probability that $ BD > 5\\sqrt{2}$?\n\n<image1>',
 'options': ['$\\frac{2 - \\sqrt{2}}{2}$',
  '$\\frac{1}{3}$',
  '$\\frac{3 - \\sqrt{3}}{3}$',
  '$\\frac{1}{2}$',
  '$\\frac{5 - \\sqrt{5}}{5}$'],
 'image': 'images/2449.jpg',
 'decoded_image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=443x316>,
 'answer': 'C',
 'solution': None,
 'level': 5,
 'subject': 'metric geometry - length'}

In [26]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import Trainer, TrainingArguments
from transformers import (
    Qwen2VLForConditionalGeneration,
    AutoTokenizer,
    AutoImageProcessor
)
from PIL import Image
import os
from typing import Dict, List

class MathVQADataset(Dataset):
    def __init__(self, data: List[Dict], tokenizer, image_processor, max_length=512, image_size=(224, 224)):
        self.data = data
        self.tokenizer = tokenizer
        self.image_processor = image_processor
        self.max_length = max_length
        self.image_size = image_size

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]

        # Load and process image
        image = Image.open(f"{img_folder_path}/{item['image']}").convert('RGB')

        # Resize the image
        image = image.resize(self.image_size)

        # Process image with minimal parameters
        image_features = self.image_processor(
            image,
            return_tensors="pt"
        )

        # Create prompt
        prompt = f"Question: {item['question']}\nAnswer:"
        target = str(item['answer'])

        # Tokenize input and target
        inputs = self.tokenizer(
            prompt,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        targets = self.tokenizer(
            target,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'labels': targets['input_ids'].squeeze(),
            'pixel_values': image_features['pixel_values'].squeeze(),
        }

def collate_fn(batch):
    input_ids = torch.stack([item['input_ids'] for item in batch])
    attention_mask = torch.stack([item['attention_mask'] for item in batch])
    labels = torch.stack([item['labels'] for item in batch])
    pixel_values = torch.stack([item['pixel_values'] for item in batch])

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': labels,
        'pixel_values': pixel_values
    }


def train_model(train_data: List[Dict], validation_data: List[Dict], output_dir: str):
    # Initialize model, tokenizer, and image processor
    model_name = "Qwen/Qwen2-VL-2B-Instruct"
    model = Qwen2VLForConditionalGeneration.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    image_processor = AutoImageProcessor.from_pretrained(model_name)

    # Create datasets
    train_dataset = MathVQADataset(train_data, tokenizer, image_processor)
    val_dataset = MathVQADataset(validation_data, tokenizer, image_processor)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=3,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=100,
        evaluation_strategy="steps",
        eval_steps=500,
        save_steps=1000,
        learning_rate=2e-5,
        fp16=True,  # Enable mixed precision training
        gradient_accumulation_steps=4,
        save_total_limit=3,
        remove_unused_columns=False
    )

    # Initialize trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=collate_fn,
    )

    # Start training
    trainer.train()

    # Save the final model
    trainer.save_model()
    tokenizer.save_pretrained(output_dir)
    image_processor.save_pretrained(output_dir)

train_model(train_data, val_data, output_dir)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


TypeError: 'NoneType' object is not iterable

In [ ]:
def generate_prompt(example, prompt_type ='full', context=''):
    question = example['question']
    options = ''
    if len(example['options']) > 0:
        assert len(example['options']) == 5, example
        if ''.join(example['options']) != 'ABCDE':
            options = f"(A) {example['options'][0]}\n(B) {example['options'][1]}\n(C) {example['options'][2]}\n(D) {example['options'][3]}\n(E) {example['options'][4]}\n"

    # input = 'Please solve the problem step by step and put your answer in one "\\boxed{}". If it is a multiple choice question, only one letter is allowed in the "\\boxed{}".\n'+f"{question}\n{options}"
    if prompt_type == 'simple':
      prompt = f"{question}\n{options}\nAnswer the question using a single word or phrase."
    else:
      prompt = f"""{context}
        Question: {question}\n{options}
        """ + 'Please solve the problem by elaborately, providing step-by-step reasoning for the solution and put your answer in one "\\boxed{}. Also, provide the bounding box coordinate of the region that can help you answer the question better.If it is a multiple choice question, only one letter is allowed in the \\boxed{}.\n'
    return prompt

In [1]:
# test

# Load the model
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_name,
    torch_dtype=torch.float32, #if device == "cpu" else torch.bfloat16,
    device_map=None
)

# Initialize processor
min_pixels = 256 * 28 * 28
max_pixels = 1280 * 28 * 28
processor = AutoProcessor.from_pretrained(model_name, min_pixels=min_pixels, max_pixels=max_pixels)


results = []

for i, input in enumerate(tqdm(test_dataset)):
    image_path = f"{img_folder_path}/{input['image']}"
    prompt = (input["question"])

    # Prepare input
    messages = [
        {"role": "user", "content": [{"type": "image", "image": image_path}, {"type": "text", "text": prompt}]}
    ]
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )

    # Move inputs to the device
    inputs = inputs.to(device)
    model = model.to(device)

    # Perform inference
    generated_ids = model.generate(**inputs, max_new_tokens=256)

    output_text = processor.batch_decode(
        generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )[0]

    # Store results
    results.append(input | {
        "prompt": prompt, "generated_text": output_text
    })
    # if i > 4:
    #   break


pd.DataFrame(results).to_csv(f"{img_folder_path}/test_generated_answer.csv")

NameError: name 'Qwen2VLForConditionalGeneration' is not defined

In [45]:
import re
remove_list = ['the answer', 'is', ':', 'boxed', '$']

results = pd.DataFrame(results)
def final_answer(text: str):
    text = text.lower()
    if 'answer is' not in text:
        return np.nan
    # Create a regex pattern to match all words in the remove_list
    pattern = r'(' + r'|'.join(re.escape(word) for word in remove_list) + r'|\s)'
    # Use re.sub to replace matched patterns with an empty string
    result = re.sub(pattern, '', text.split('answer is')[-1])
    result = re.sub(r'\\\{(\\frac\{[^{}]+\}\{[^{}]+\})\}', r'\1', result)
    return re.sub(r'\\\\', r'\\', result)
# Example metric: String matching (very basic)
results['prediction'] = results['generated_text'].apply(final_answer)
results['exact_match'] = results['prediction'] == results['answer']
display(results)
pd.DataFrame(results).to_csv(f"{img_folder_path}/test_generated_answer.csv")

accuracy = sum(results['exact_match']) / len(results)
print(f"Exact Match Accuracy: {accuracy:.4f}")

# Optionally save results
# import json
# with open("evaluation_results.json", "w") as f:
#     json.dump(results, f, indent=4)

,id,question,options,image,decoded_image,answer,solution,level,subject,prompt,generated_text,prediction,exact_match
0,2941,A sphere is inscribed in a cone with height 4 ...,[],images/2941.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...,\frac{3}{8},We start by finding the radius of the inscribe...,2,solid geometry,A sphere is inscribed in a cone with height 4 ...,system\nYou are a helpful assistant.\nuser\nA ...,\frac{3}{1}.,False
1,2942,The length of the diameter of this spherical b...,[],images/2942.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...,27000-4500\pi,The box has volume $30^3=27000$ cubic cm.\n\nT...,1,solid geometry,The length of the diameter of this spherical b...,system\nYou are a helpful assistant.\nuser\nTh...,NaN,False
2,2943,"In the circle below, $\overline{AB} \| \overli...",[],images/2943.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...,8\pi,"By symmetry, $\widehat{BD}=\widehat{CA}=100^\c...",4,metric geometry - length,"In the circle below, $\overline{AB} \| \overli...",system\nYou are a helpful assistant.\nuser\nIn...,NaN,False
3,2944,"In right triangle $ABC$, $\angle B = 90^\circ$...",[],images/2944.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...,2\sqrt{3},"Let $\overline{DE}$ have length $x$, so $\over...",4,metric geometry - length,"In right triangle $ABC$, $\angle B = 90^\circ$...",system\nYou are a helpful assistant.\nuser\nIn...,NaN,False
4,2945,The area of square $ABCD$ is 100 square centim...,[],images/2945.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...,68,"Since $AE = 2$, $EB = 8$, but since $EFGH$ is ...",1,metric geometry - area,The area of square $ABCD$ is 100 square centim...,system\nYou are a helpful assistant.\nuser\nTh...,100,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3036,The two externally tangent circles each have a...,[],images/3036.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...,8-2\pi,Each diameter of a circle is 2 units. The rect...,1,metric geometry - area,The two externally tangent circles each have a...,system\nYou are a helpful assistant.\nuser\nTh...,NaN,False
96,3037,The area of $\triangle ABC$ is 6 square centim...,[],images/3037.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...,54,"Since $AB \parallel DE,$ we know that $\angle ...",4,metric geometry - area,The area of $\triangle ABC$ is 6 square centim...,system\nYou are a helpful assistant.\nuser\nTh...,NaN,False
97,3038,"In the diagram, $K$, $O$ and $M$ are the cente...",[],images/3038.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1250\pi,We know that $OA$ and $OB$ are each radii of t...,4,metric geometry - area,"In the diagram, $K$, $O$ and $M$ are the cente...",system\nYou are a helpful assistant.\nuser\nIn...,512\pi,False
98,3039,The volume of the cylinder shown is $45\pi$ cu...,[],images/3039.jpg,<PIL.JpegImagePlugin.JpegImageFile image mode=...,5,The volume of the cylinder is $bh=\pi r^2h$. T...,1,solid geometry,The volume of the cylinder shown is $45\pi$ cu...,system\nYou are a helpful assistant.\nuser\nTh...,NaN,False


Exact Match Accuracy: 0.0200
